In [15]:
import torch
x=torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [16]:
x.requires_grad_(True)#只有张量的requires_grad属性为True，PyTorch 才会在执行反向传播时计算该张量对于损失函数的梯度
x.grad
#在 PyTorch 中，x.grad 是一个张量（tensor），它保存着张量 x 相对于某个标量损失函数的梯度。
#具体来说，如果 y 是一个标量张量，且通过执行反向传播计算出了 y 对于 x 的梯度，那么可以通过 x.grad 获取这个梯度值。
# 如果张量 x 的 requires_grad 属性为 False，那么 x.grad 的值为 None，因为 PyTorch 不会对不需要计算梯度的张量进行梯度计算。
# 另外，如果需要多次计算梯度，需要在每次计算之前将 x.grad 清零，否则 PyTorch 会累加每次计算的梯度值。


In [17]:
y=2*torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

In [18]:
y.backward()  # 调用反向传播函数来自动计算y关于x每个分量的梯度
x.grad

tensor([ 0.,  4.,  8., 12.])

In [19]:
#函数y=2*xTx关于x的梯度应该是4x,看是否梯度求对
x.grad==4*x


tensor([True, True, True, True])

In [20]:
x.grad.zero_()#清零梯度
y=x.sum()#y的表达式从新定义为y=x1+x2+x3+x4
y.backward()  # 进行反向传播计算y关于x每个分量的梯度，因为xi梯度是1，所以算出来的梯度向量肯定是[1,1,1,1]
x.grad

tensor([1., 1., 1., 1.])

In [21]:
x*x

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

In [22]:
# 非标量变量的反向传播
#在 PyTorch 中，反向传播算法要求损失函数是一个标量（scalar），因为只有标量才有一个唯一的梯度值。
#如果损失函数是一个非标量（如矩阵或张量），那么我们需要先将它转化为一个标量，才能进行反向传播
# 常见的方法是对损失函数进行求和或平均，将其变为标
x.grad.zero_()
y=x*x#得出来的是一个向量，不是标量
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

# 分离计算
### 计算图：计算图（computational graph）是指记录了张量之间计算依赖关系的有向无环图。在计算图中，每个节点代表一个张量或者一个操作，每个边代表依赖
在 PyTorch 中，每个张量都有一个 grad_fn 属性，表示创建该张量的操作。当我们进行前向传播时，PyTorch 会自动构建计算图，并将计算图中的每个操作关联到其输出张量的 grad_fn 属性上。这样，在进行反向传播时，PyTorch 就可以使用这个计算图来自动计算梯度。
有时候，我们希望将某些计算移动到记录的计算图之外，以便能够在反向传播时忽略这些计算，或者用一些其他方式处理它们
假设y是作为x的函数计算的，而z则是作为y和x的函数计算的。
想象一下，我们想计算z关于x的梯度，但由于某种原因，希望将y视为一个常数， 并且只考虑到x在y被计算后发挥的作用。

In [24]:
# 在 PyTorch 中，detach() 方法可以将张量从计算图中分离出来，返回一个新的张量，这个新的张量和原来的张量在数值上是相等的，但是在计算图中已经不再依赖原来的张量。
# 因此，对分离后的张量进行操作不会影响原来的张量的梯度计算。
x.grad.zero_()
y=x*x
u=y.detach()#此时u已经从计算图中分离出来
z=u*x#此时的u不再是变量而只是一个常量
z.sum().backward()
x.grad==u#z对x的梯度即是u

tensor([True, True, True, True])

In [25]:
x.grad.zero_()
y.sum().backward()
x.grad==2*x

tensor([True, True, True, True])

In [47]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [55]:
a = torch.randn(size=(), requires_grad=True)
d=f(a)
k=f(a)
d.backward()
a.grad==d/a#在 PyTorch 中，对于一个标量张量，可以使用除法操作得到一个标量值。d/a 的含义是f'(a)，即函数 f 在 a 处的导数
# 注意d是做了反向传播的，所以这个d/a才能看作是求d在a处的倒数（特定情形适用）

tensor(True)